In [78]:
import ots_backend as backend
from json import loads
import pandas as pd
from requests import get

In [66]:
ncaaf = pd.concat([pd.read_csv('../database/ncaaf_2023.csv'), pd.read_csv('../database/ncaaf_2023.csv')])

In [87]:
ncaaf['date'] = None
ncaaf['venue'] = None
ncaaf['stadium_indoor'] = None
ncaaf['city'] = None
ncaaf['state'] = None
ncaaf['zip'] = None
ncaaf['grass'] = None
ncaaf['attendance'] = None

ncaaf['away_id'] = None
ncaaf['away_school'] = None
ncaaf['away_name'] = None
ncaaf['away_abbreviation'] = None
ncaaf['away_color'] = None

ncaaf['home_id'] = None
ncaaf['home_school'] = None
ncaaf['home_name'] = None
ncaaf['home_abbreviation'] = None
ncaaf['home_color'] = None
ncaaf['neutral_site'] = None

for i in range(len(ncaaf)):
    extract_main = loads(ncaaf['extract_main'].values[i])
    extract_from_url = loads(ncaaf['extract_from_url'].values[i])
    
    ncaaf['date'].values[i] = extract_main.get('header').get('competitions')[0].get('date')
    ncaaf['venue'].values[i] = extract_main.get('gameInfo').get('venue').get('fullName')
    ncaaf['stadium_indoor'].values[i]  = extract_from_url.get('competitions')[0].get('venue').get('indoor')
    ncaaf['city'].values[i] = extract_from_url.get('competitions')[0].get('venue').get('address').get('city')
    ncaaf['state'].values[i] = extract_from_url.get('competitions')[0].get('venue').get('address').get('state')
    ncaaf['zip'].values[i] = extract_from_url.get('competitions')[0].get('venue').get('address').get('zipCode')
    ncaaf['grass'].values[i] = extract_main.get('gameInfo').get('venue').get('grass')
    ncaaf['attendance'].values[i] = extract_main.get('gameInfo').get('attendance')

    ncaaf['away_id'].values[i] = extract_main.get('boxscore').get('teams')[0].get('team').get('id', None)
    ncaaf['away_school'].values[i] = extract_main.get('boxscore').get('teams')[0].get('team').get('location', None)
    ncaaf['away_name'].values[i] = extract_main.get('boxscore').get('teams')[0].get('team').get('name', None)
    ncaaf['away_abbreviation'].values[i] = extract_main.get('boxscore').get('teams')[0].get('team').get('abbreviation', None)
    ncaaf['away_color'].values[i] = extract_main.get('boxscore').get('teams')[0].get('team').get('color', None)

    ncaaf['home_id'].values[i] = extract_main.get('boxscore').get('teams')[1].get('team').get('id', None)
    ncaaf['home_school'].values[i] = extract_main.get('boxscore').get('teams')[1].get('team').get('location', None)
    ncaaf['home_name'].values[i] = extract_main.get('boxscore').get('teams')[1].get('team').get('name', None)
    ncaaf['home_abbreviation'].values[i] = extract_main.get('boxscore').get('teams')[1].get('team').get('abbreviation', None)
    ncaaf['home_color'].values[i] = extract_main.get('boxscore').get('teams')[1].get('team').get('color', None)

    ncaaf['neutral_site'].values[i] = extract_from_url.get('competitions')[0].get('neutralSite')

Teams = ncaaf[['home_id', 'home_school', 'home_name', 'home_abbreviation', 'home_color', 'venue', 'city', 'state', 'zip', 'grass', 'stadium_indoor', 'neutral_site']]
Teams = Teams.rename(columns = {'home_id':'espn_id', 'home_school':'school', 'home_name':'name', 'home_abbreviation':'abbreviation', 'home_color':'color', 'venue':'stadium'})
Teams_count = Teams.groupby('school').count().sort_values('espn_id').reset_index(drop=False)
Teams_count = Teams_count.rename(columns = {'espn_id':'espn_id_count'})
Teams_count = Teams_count[['school', 'espn_id_count']]
Teams = Teams.merge(Teams_count, how = 'left', on = 'school')
Teams = Teams[Teams['espn_id_count'] > 5]
Teams = Teams[Teams['neutral_site'] == False]
Teams = Teams.drop_duplicates(subset=['school'])
Teams = Teams.drop(columns=['espn_id_count', 'neutral_site'])
Teams['division'] = None

In [88]:
for i in range(len(Teams)):
    espn_id = Teams['espn_id'].values[i]
    res = get('https://sports.core.api.espn.com/v2/sports/football/leagues/college-football/seasons/2023/types/3/groups/' + str(espn_id) + '?lang=en&region=us')
    res = loads(res.content.decode('utf-8'))
    Teams['division'].values[i] = res.get('name')

'Mountain West Conference'

,espn_id,school,name,abbreviation,color,stadium,city,state,zip,grass,stadium_indoor
0,30,USC,Trojans,USC,9e2237,Los Angeles Memorial Coliseum,Los Angeles,CA,90037,True,False
2,55,Jacksonville State,Gamecocks,JVST,b50500,Burgess-Snow Field,Jacksonville,AL,36265,False,False
3,166,New Mexico State,Aggies,NMSU,891216,Aggie Memorial Stadium,Las Cruces,NM,88003,False,False
4,21,San Diego State,Aztecs,SDSU,c41230,Snapdragon Stadium,San Diego,CA,92108,False,False
5,2348,Louisiana Tech,Bulldogs,LT,002d65,Joe Aillet Stadium,Ruston,LA,71270,False,False
...,...,...,...,...,...,...,...,...,...,...,...
233,158,Nebraska,Cornhuskers,NEB,d00000,"Memorial Stadium (Lincoln, NE)",Lincoln,NE,68508,False,False
250,24,Stanford,Cardinal,STAN,8c1515,Stanford Stadium,Stanford,CA,94305,True,False
253,326,Texas State,Bobcats,TXST,4e1719,Bobcat Stadium (TX),San Marcos,TX,78666,False,False
299,193,Miami (OH),RedHawks,M-OH,a4000c,Yager Stadium,Oxford,OH,45056,False,False


In [77]:



# res.get('name')

{'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/college-football/seasons/2023/types/3/groups/17?lang=en&region=us',
 'uid': 's:20~l:23~g:17',
 'id': '17',
 'name': 'Mountain West Conference',
 'abbreviation': 'mwest',
 'shortName': 'Mountain West',
 'midsizeName': 'Mountain West',
 'season': {'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/college-football/seasons/2023?lang=en&region=us'},
 'parent': {'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/college-football/seasons/2023/types/3/groups/80?lang=en&region=us'},
 'standings': {'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/college-football/seasons/2023/types/3/groups/17/standings?lang=en&region=us'},
 'isConference': True,
 'logos': [{'href': 'https://a.espncdn.com/i/teamlogos/ncaa_conf/500/mountain_west.png'}],
 'slug': 'mountain-west',
 'teams': {'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/college-football/seasons/2023/types/

In [4]:
game.keys()

dict_keys(['boxscore', 'format', 'gameInfo', 'drives', 'leaders', 'broadcasts', 'predictor', 'pickcenter', 'againstTheSpread', 'odds', 'winprobability', 'header', 'scoringPlays', 'news', 'videos', 'standings'])

In [21]:
game.get('header').get('competitions')[0].get('neutralSite')

True

In [173]:
away_stats = pd.json_normalize(game.get('boxscore').get('teams')[0].get('statistics'))
away_stats = away_stats[['name', 'displayValue']].reset_index(drop = True)
away_stats = away_stats.rename(columns = {'displayValue': 'away'})

home_stats = pd.json_normalize(game.get('boxscore').get('teams')[1].get('statistics'))
home_stats = home_stats[['name', 'displayValue']].reset_index(drop = True)
home_stats = home_stats.rename(columns = {'displayValue': 'home'})

stats_template = pd.DataFrame({
    'name': ['firstDowns', 'thirdDownEff', 'fourthDownEff', 'totalYards',
       'netPassingYards', 'completionAttempts', 'yardsPerPass', 'rushingYards',
       'rushingAttempts', 'yardsPerRushAttempt', 'totalPenaltiesYards',
       'turnovers', 'fumblesLost', 'interceptions', 'possessionTime']
})

stats_template = stats_template.merge(away_stats, how = 'left', on = 'name').merge(home_stats, how = 'left', on = 'name')
stats_template

,name,away,home
0,firstDowns,17,16
1,thirdDownEff,2-14,1-10
2,fourthDownEff,2-5,0-1
3,totalYards,301,443
4,netPassingYards,255,140
5,completionAttempts,27-51,10-19
6,yardsPerPass,5.0,7.4
7,rushingYards,46,303
8,rushingAttempts,20,38
9,yardsPerRushAttempt,2.3,8.0


In [159]:
stats = pd.json_normalize(game.get('boxscore').get('teams')[0].get('statistics'))
stats = stats[['name', 'displayValue']].reset_index(drop = True)
stats = stats.transpose()
stats.columns = stats.iloc[0, ]
stats = stats.drop(['name'])
stats.columns.name = 'index'
stats.reset_index(drop = True)

index,firstDowns,thirdDownEff,fourthDownEff,totalYards,netPassingYards,completionAttempts,yardsPerPass,rushingYards,rushingAttempts,yardsPerRushAttempt,totalPenaltiesYards,turnovers,fumblesLost,interceptions,possessionTime
0,17,2-14,2-5,301,255,27-51,5.0,46,20,2.3,5-30,2,0,2,30:44


In [160]:
stats.columns

Index(['firstDowns', 'thirdDownEff', 'fourthDownEff', 'totalYards',
       'netPassingYards', 'completionAttempts', 'yardsPerPass', 'rushingYards',
       'rushingAttempts', 'yardsPerRushAttempt', 'totalPenaltiesYards',
       'turnovers', 'fumblesLost', 'interceptions', 'possessionTime'],
      dtype='object', name='index')

index,firstDowns,thirdDownEff,fourthDownEff,totalYards,netPassingYards,completionAttempts,yardsPerPass,rushingYards,rushingAttempts,yardsPerRushAttempt,totalPenaltiesYards,turnovers,fumblesLost,interceptions,possessionTime
1,17,2-14,2-5,301,255,27-51,5.0,46,20,2.3,5-30,2,0,2,30:44


In [32]:
# Meta
date = raw.get('header').get('competitions')[0].get('date')

# Away
away_id = raw.get('boxscore').get('teams')[0].get('team').get('id', None)
away_school = raw.get('boxscore').get('teams')[0].get('team').get('location', None)
away_name = raw.get('boxscore').get('teams')[0].get('team').get('name', None)
away_abbreviation = raw.get('boxscore').get('teams')[0].get('team').get('abbreviation', None)
away_color = raw.get('boxscore').get('teams')[0].get('team').get('color', None)

# Home
home_id = raw.get('boxscore').get('teams')[1].get('team').get('id', None)
home_school = raw.get('boxscore').get('teams')[1].get('team').get('location', None)
home_name = raw.get('boxscore').get('teams')[1].get('team').get('name', None)
home_abbreviation = raw.get('boxscore').get('teams')[1].get('team').get('abbreviation', None)
home_color = raw.get('boxscore').get('teams')[1].get('team').get('color', None)

In [34]:
raw.get('boxscore').get('teams')[0]

{'team': {'id': '264',
  'uid': 's:20~l:23~t:264',
  'slug': 'washington-huskies',
  'location': 'Washington',
  'name': 'Huskies',
  'abbreviation': 'WASH',
  'displayName': 'Washington Huskies',
  'shortDisplayName': 'Washington',
  'color': '33006f',
  'logo': 'https://a.espncdn.com/i/teamlogos/ncaa/500/264.png'},
 'statistics': [{'name': 'firstDowns',
   'displayValue': '17',
   'label': '1st Downs'},
  {'name': 'thirdDownEff',
   'displayValue': '2-14',
   'label': '3rd down efficiency'},
  {'name': 'fourthDownEff',
   'displayValue': '2-5',
   'label': '4th down efficiency'},
  {'name': 'totalYards', 'displayValue': '301', 'label': 'Total Yards'},
  {'name': 'netPassingYards', 'displayValue': '255', 'label': 'Passing'},
  {'name': 'completionAttempts', 'displayValue': '27-51', 'label': 'Comp-Att'},
  {'name': 'yardsPerPass', 'displayValue': '5.0', 'label': 'Yards per pass'},
  {'name': 'rushingYards', 'displayValue': '46', 'label': 'Rushing'},
  {'name': 'rushingAttempts',
   'di

In [31]:
away_id = raw.get('boxscore').get('teams')[0].get('team').get('id', None)
away_school = raw.get('boxscore').get('teams')[0].get('team').get('location', None)
away_name = raw.get('boxscore').get('teams')[0].get('team').get('name', None)
away_abbreviation = raw.get('boxscore').get('teams')[0].get('team').get('abbreviation', None)
away_color = raw.get('boxscore').get('teams')[0].get('team').get('color', None)

In [1]:
import requests

In [2]:
res = requests.get('http://site.api.espn.com/apis/site/v2/sports/football/college-football/summary?event=401551789')

In [3]:
res.content

b'{"boxscore":{"teams":[{"team":{"id":"264","uid":"s:20~l:23~t:264","slug":"washington-huskies","location":"Washington","name":"Huskies","abbreviation":"WASH","displayName":"Washington Huskies","shortDisplayName":"Washington","color":"33006f","logo":"https://a.espncdn.com/i/teamlogos/ncaa/500/264.png"},"statistics":[{"name":"firstDowns","displayValue":"17","label":"1st Downs"},{"name":"thirdDownEff","displayValue":"2-14","label":"3rd down efficiency"},{"name":"fourthDownEff","displayValue":"2-5","label":"4th down efficiency"},{"name":"totalYards","displayValue":"301","label":"Total Yards"},{"name":"netPassingYards","displayValue":"255","label":"Passing"},{"name":"completionAttempts","displayValue":"27-51","label":"Comp-Att"},{"name":"yardsPerPass","displayValue":"5.0","label":"Yards per pass"},{"name":"rushingYards","displayValue":"46","label":"Rushing"},{"name":"rushingAttempts","displayValue":"20","label":"Rushing Attempts"},{"name":"yardsPerRushAttempt","displayValue":"2.3","label":